In [1]:
import csv
import boto3

In [2]:
## set-up for AWS  
s3_client = boto3.client('s3')  
bucket_name = 'ca-climate-index'  
directory = '2a_subset/climate_risk/flood/loss/fema/flood_claims_ca'

In [4]:
def filter_and_save_csv(input_file_name, output_file_name):
    '''
    Takes FEMA's flood claim data and subsets for California claims exclusively
    
    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are stored in ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    
    Parameters
    ----------
    input_file_name: string
              The name of the file containing FEMA's Flood Claim data: https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2
    output_file_name: string
              Final output as a .csv file
    '''
    with open(input_file_name, 'r', newline='') as input_file:
        # Create a CSV reader
        csv_reader = csv.DictReader(input_file)

        # Extract the header from the reader
        header = csv_reader.fieldnames

        # Create a list to store filtered rows
        filtered_rows = [row for row in csv_reader if row['state'] == 'CA']

    # Open the output CSV file for writing
    with open(output_file_name, 'w', newline='') as output_file:
        # Create a CSV writer
        csv_writer = csv.DictWriter(output_file, fieldnames=header, delimiter = ',')

        # Write the header to the output file
        csv_writer.writeheader()

        # Write the filtered rows to the output file
        csv_writer.writerows(filtered_rows)

# Example usage:
input_csv = 'FimaNfipClaims.csv'
output_csv = 'fema_claims_CA_subset.csv'

filter_and_save_csv(input_csv, output_csv)

# Save the file to AWS S3 using the client
with open(output_csv, 'rb') as data:
    s3_client.upload_fileobj(data, bucket_name, f"{directory}/{output_csv}")
print(f"Merged and sorted files written to {output_csv}")

Merged and sorted files written to fema_claims_CA_subset.csv
